In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
def clean_text(df):
    for i, t in enumerate(df['Testo']):
        if len(t) < 4:
            df['Testo'][i] = ''
    return df

def fill_null_features(df):
    for c in ['Domanda','Testo']:
        for i in range(0,len(df.index)):  
            if not df[c][i]:
                j=i
                while j>0: 
                    j-=1
                    if df[c][j]:
                        df[c][i] = df[c][j]
                        break
    return df

# Se lo stralcio è vuoto, prendo tutto il testo come stralcio. Non usata su Hyperion
def copy_text(df):
    for i in range(0,len(df.index)):  
            if not df['Stralcio'][i]:
                df['Stralcio'][i] = df['Testo'][i]
    return df

def add_context(df):
    context_list = []
    for i, span in enumerate(df['Testo']):
        
        # first row
        if i == 0:
            context =span + ' ' + df['Stralcio'][i+1]
        #last row
        elif i == len(df.index)-1:
            context = df['Stralcio'][i-1] + ' ' + span
        
        #text with only 1 span
        elif df['Testo'][i] and df['Testo'][i+1]:
            context = span
        #first span
        elif df['Testo'][i]:
            context = span + ' ' + df['Stralcio'][i+1]
        #last span
        elif df['Testo'][i+1]:
            context = df['Stralcio'][i-1] + ' ' + span
        
        else:
            context = df['Stralcio'][i-1] + ' ' + span + ' ' + df['Stralcio'][i+1]
        context_list.append(context)
    df['Contesto']= context_list
    print(len(context_list))
    return df

def uniform_labels(df):
    df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
    df['Repertorio'].replace('previsioni','previsione', inplace=True)



In [3]:
#Hyperion dataset
df = pd.read_csv('../data/raw/original_csv/hyperion.csv', na_filter=False)
df = clean_text(df)
#df = add_context(df)
df['Repertorio'] = df['Repertorio'].str.lower()
uniform_labels(df)


print('Numero stralci: ' + str(len(df.index)))
filter = df["Repertorio"] != ""
df = df[filter]
#df = df.applymap(str.lower)
print('Numero stralci dopo eliminazione: ' + str(len(df.index)))


Numero stralci: 35474
Numero stralci dopo eliminazione: 35148


In [4]:
df.loc[0]

Domanda       A cosa serve questo testo per il perseguimento...
Testo         Dunque vediamo se ho capito: Conte le canta a ...
Stralcio                           Dunque vediamo se ho capito:
Repertorio                             dichiarazione di intenti
Name: 0, dtype: object

In [5]:
df.to_csv ('../data/processed/clean/hyperion.csv', index = False, header=True )

## Splitting dataset to evaluate the entire pipeline

In [6]:
df = pd.read_csv('../data/processed/clean/hyperion.csv')

In [7]:
dataset = []

for row in df.itertuples():
    text = row.Testo
    
    if pd.isna(text):
        sample['Stralci'].append(row.Stralcio)
        sample['Repertori'].append(row.Repertorio)

    else:
        sample = {}
        sample['Testo'] = text
        sample['Stralci'] = [row.Stralcio]
        sample['Repertori'] = [row.Repertorio]
        dataset.append(sample)

IE_dict = {
                'Testo': [sample['Testo'] for sample in dataset],
                'Stralci': [sample['Stralci'] for sample in dataset],
                'Repertori': [sample['Repertori'] for sample in dataset]

            }
df = pd.DataFrame(IE_dict)

train_size = 0.8
train_df = df.sample(frac=train_size, random_state=200)
test_df = df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

train_df.to_csv ('../data/processed/pipeline/train/ie_hyperion.csv', index = False, header=True)
test_df.to_csv ('../data/processed/pipeline/test/ie_hyperion.csv', index = False, header=True)

FULL Dataset: (15332, 3)
TRAIN Dataset: (12266, 3)
TEST Dataset: (3066, 3)


In [26]:
dataset = []

for row in train_df.itertuples():
    sample = {}
    sample['Testo'] = row.Testo
    sample['Stralcio'] = row.Stralci[0]
    sample['Repertorio'] = row.Repertori[0]
    dataset.append(sample)

    for i in range(1,len(row.Stralci)):
        sample = {}
        sample['Testo'] = ''
        sample['Stralcio'] = row.Stralci[i]
        sample['Repertorio'] = row.Repertori[i]
        dataset.append(sample)

IE_dict = {
                'Testo': [sample['Testo'] for sample in dataset],
                'Stralcio': [sample['Stralcio'] for sample in dataset],
                'Repertorio': [sample['Repertorio'] for sample in dataset]

            }
df = pd.DataFrame(IE_dict)

df.to_csv ('../data/processed/pipeline/train/hyperion.csv', index = False, header=True)




In [31]:
dataset = []

for row in test_df.itertuples():
    sample = {}
    sample['Testo'] = row.Testo
    sample['Stralcio'] = row.Stralci[0]
    sample['Repertorio'] = row.Repertori[0]
    dataset.append(sample)

    for i in range(1,len(row.Stralci)):
        sample = {}
        sample['Testo'] = ''
        sample['Stralcio'] = row.Stralci[i]
        sample['Repertorio'] = row.Repertori[i]
        dataset.append(sample)

IE_dict = {
                'Testo': [sample['Testo'] for sample in dataset],
                'Stralcio': [sample['Stralcio'] for sample in dataset],
                'Repertorio': [sample['Repertorio'] for sample in dataset]

            }
df = pd.DataFrame(IE_dict)

df.to_csv ('../data/processed/pipeline/test/hyperion.csv', index = False, header=True)

## Splitting balanced dataset for training repertoires classifier

In [32]:
df = pd.read_csv('../data/processed/pipeline/train/hyperion.csv', na_filter=False)

In [33]:
df.groupby('Repertorio')['Repertorio'].count()

Repertorio
anticipazione                 38
causa                        377
commento                    3496
conferma                     502
considerazione               238
contrapposizione            1141
deresponsabilizzazione       521
descrizione                 3900
dichiarazione di intenti     507
generalizzazione             841
giudizio                    2191
giustificazione              360
implicazione                 558
non risposta                 916
opinione                    1035
possibilità                  487
prescrizione                1540
previsione                   929
proposta                     216
ridimensionamento            856
sancire                     4488
specificazione              1094
valutazione                 1678
Name: Repertorio, dtype: int64

# Split

In [35]:
# split
test_set_perc = 0.2

gb = df.groupby('Repertorio')
train_list = []
test_list = []

for x in gb.groups:
    class_df = gb.get_group(x)

    # Test set creation
    test = class_df.sample(frac=test_set_perc, random_state=1464)
    train = pd.concat([class_df,test]).drop_duplicates(keep=False)

    train_list.append(train)
    test_list.append(test)

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

train_df.to_csv ('../data/processed/splitted/hyperion_train.csv', index = False, header=True)
test_df.to_csv ('../data/processed/splitted/hyperion_test.csv', index = False, header=True)  

In [36]:
train_df.groupby('Repertorio')['Repertorio'].count()

Repertorio
anticipazione                 30
causa                        300
commento                    2778
conferma                     401
considerazione               190
contrapposizione             908
deresponsabilizzazione       416
descrizione                 3108
dichiarazione di intenti     402
generalizzazione             665
giudizio                    1749
giustificazione              286
implicazione                 445
non risposta                 733
opinione                     823
possibilità                  390
prescrizione                1230
previsione                   736
proposta                     173
ridimensionamento            669
sancire                     3570
specificazione               874
valutazione                 1342
Name: Repertorio, dtype: int64